### Computing Platform Check GPU (CUDA) or CPU

In [ ]:
import torch
if torch.cuda.is_available():
    device = "cuda"
else:
    print ('[WARNING] CUDA/GPU is not available! Compute-intensive scripts on this notebook will be run on CPU.')
    device =  "cpu"

## Check Environment

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    print("Environment is colab")
elif 'KAGGLE_URL_BASE' in os.environ:
    print("Environment is kaggle")
else:
    print("Environment is local")

In [ ]:
#@title Importing open_clip
try : 
  import open_clip
except ImportError:
  !pip install open_clip_torch
  import open_clip

In [ ]:
#@title Other Imports

# Handling all importing error 
try:
  import torch
except ImportError:
  !pip install torch>=1.9.0
  import torch

try:
  import PIL
  from PIL import Image
except:
  !pip install pillow
  import PIL
  from PIL import Image

In [ ]:
#@title Available CLIP Models
# prints all the avaialble clip models
# Choose one of these models :: model name and pretrained 
open_clip.list_pretrained()

In [ ]:
#@title Loading CLIP model
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L/14',pretrained='openai')

In [ ]:
#@title Run Example
import pathlib

image_path = pathlib.Path.cwd().parent / 'images' / 'download_blue.png'
txt_list = ["pixel art" , "photography"]

image = preprocess(Image.open(image_path)).unsqueeze(0)
text = open_clip.tokenize(txt_list)

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    # .T for transpose 
    # @ : for matrix multiplication 
    # Calculating cosing similarity between text and image features 
    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    # Getting the top similar image text feature to the image using .argmax()
    # which outputs the index of the maximum value in the text_probs tensor 
    print(f'Matched Text :: { txt_list[text_probs.argmax()] } ')

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]